## <u> 맥북에서 작성한 코드 </u>

In [6]:
import pandas as pd
df = pd.read_csv('당근크롤링.csv')
print(df.shape)
df = df.drop_duplicates(subset=['article_id'])
print(df.shape)
df.to_csv('당근drop_duplitcate.csv',index=False,encoding='utf-8-sig')

(1180, 8)
(1127, 8)


In [5]:
df['article_id'].nunique()

1127

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
article_id_list = [] # /articles/{articles_id}
try :
    url = 'https://www.daangn.com/search/%EB%A7%A5%EB%B6%81'
    driver = webdriver.Chrome(executable_path='/Users/june/Desktop/danggn/chromedriver')
    # driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.implicitly_wait(3)
    count = 0 
    while True : 
        count += 1
        if count == 100 : break
        try : 
            driver.find_element(By.CSS_SELECTOR, '#result > div:nth-child(1) > div.more-btn').click()
            time.sleep(2)
        except Exception as e:
            print(f'에러내역 : {e}')
            break
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    article_id = soup.find_all('a',attrs={'class':'flea-market-article-link'})
    print('======================')
    print('결과값')
    for c, id in enumerate(article_id) :
        article_id_list += [id['href'][10:]]
        print(c, id['href'][10:])
finally :
    driver.quit()

/var/folders/sd/c66x6jh14991sn8g9gv1f3x80000gn/T/ipykernel_11222/1370891690.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/june/Desktop/danggn/chromedriver')


결과값
0 464885104
1 451688581
2 475620502
3 475617431
4 474581097
5 463875775
6 451626950
7 437094806
8 475385326
9 463802846
10 475340234
11 475438836
12 475416352
13 469759011
14 467543728
15 464109675
16 475308079
17 474148053
18 469134671
19 470819227
20 475050953
21 441939781
22 466803413
23 445026389
24 466842778
25 473085921
26 473709426
27 467208356
28 472292418
29 474643745
30 468066321
31 468668519
32 471574503
33 472003052
34 473432609
35 472002218
36 463378856
37 474541691
38 435012938
39 472790082
40 474339126
41 460296071
42 471351101
43 319128698
44 435545918
45 472369322
46 466465212
47 455949988
48 474188931
49 456434240
50 469466909
51 467540760
52 474194765
53 474092324
54 470413410
55 458822750
56 465673715
57 346713570
58 473970077
59 473851125
60 472812956
61 458197006
62 465499291
63 415201797
64 444793750
65 470198187
66 470922814
67 465119289
68 465370911
69 473432325
70 463523083
71 472078446
72 466295163
73 460887834
74 473432911
75 464322745
76 467650481
77 45

### 소스코드에 이런 것들도 있는데 왜 있는지 모르겠음

meta content="가격" name="twitter:label1"/>
meta content="350,000원" name="twitter:data1"/>
meta content="동네" name="twitter:label2"/>
meta content="서울 강서구 등촌동" name="twitter:data2"/>
meta content="디지털기기" name="product:category"/>
meta content="used" name="product:condition"/>
meta content="instock" name="product:availability"/>
meta content="350,000원" name="product:price:amount"/>
meta content="KRW" name="product:price:currency"/>
meta content="350,000원" name="product:sale_price:amount"/>
meta content="KRW" name="product:sale_price:currency"/>

In [10]:
# article_id 까지 해서 관계형 테이블로 만들면 됨
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['article_id','title','region','written_date','manner_temperature','price','content','chat_count'])

try :
    driver = webdriver.Chrome(executable_path='/Users/june/Desktop/danggn/chromedriver')
    for c, article_id in enumerate(article_id_list) :
        url = f'https://www.daangn.com/articles/{article_id}'
        driver.get(url)
        # driver.implicitly_wait(3)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # print(soup.prettify) # 전체 소스코드 훑어보기
        if soup.find('p',attrs={'id':'no-article'}) is not None :
            df.loc[len(df)] = [article_id,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
            continue
        ##### parsing
        
        title = soup.find('h1',attrs={'id':'article-title'}).get_text() # <h1 id="article-title" property="schema:name" style="margin-top:0px;">맥북 프로 MacBook Pro 13(Retina, 2013년 후반 모델)</h1>
        content = soup.find('div',attrs={'id':'article-detail'}).get_text()[1:-1] # <div id="article-detail" property="schema:description">
        region = soup.find('div', attrs={'id':'region-name'}).get_text() # <div id="region-name">강서구 등촌동</div>
        written_date = soup.find('time').get_text().replace(' ','').replace('\n','') # <section id="article-description"> <p id="article-category"> 디지털기기 ∙ <time> 끌올 6시간 전 </time> </p>
        manner_temperature = soup.select_one('#temperature-wrap > dd').get_text().replace(' ','').replace('\n','')
        try : 
            price = soup.find('p',attrs={'id':'article-price'}).get_text().replace(' ','').replace('\n','') # <p content="350000.0" id="article-price" property="schema:price" style="font-size:18px; font-weight:bold;"> 350,000원 </p>
        except :
            price = soup.find('p',attrs={'id':'article-price-nanum'}).get_text().replace(' ','').replace('\n','')
        chat_count = soup.find('p',attrs={'id':'article-counts'}).get_text().replace(' ','').replace('\n','').replace('∙','|') # <p id="article-counts">
        try :
            image_url = soup.select_one('#slick-slide00 > div > a').attrs['href']
            if image_url is None :
                image_url = soup.select_one('#image-slider > div > div > div > div > div > div > a').attrs['href']
        except :
            print(f'이미지없음 {article_id}')
        print(title)
        print(image_url)
        print(region)
        print(written_date)
        print(manner_temperature)
        print(price)
        print(content)
        print(chat_count)
        print(f'{c}=================================================')
        
        df.loc[len(df)] = [article_id,title,region,written_date,manner_temperature,price,content,chat_count]


finally :
    driver.quit()

/var/folders/sd/c66x6jh14991sn8g9gv1f3x80000gn/T/ipykernel_11222/3793479135.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/june/Desktop/danggn/chromedriver')


맥북프로 13인치 2012 late 레티나 (Macbook Pro)
/images?image_index=0&object_id=464885104&object_type=Article
강남구 청담동
끌올1시간전
39.8°C
230,000원
맥북프로 13인치 2012 late 레티나 모델입니다.
홍콩에서 구입 후 집에서만 사용 후 보관하였습니다. (한국 전원코드 있음)
​
보시면 아시겠지만 집에서만 사용했어서 외관 깔끔하며 배터리 사이클도 적은 편입니다.
스펙은 i5 2.5Ghz, 8GB Ram, 256GB SSD 탑재한 모델입니다.
​
배터리 사이클은 124회이며 최대충전용량운 4825mAh입니다. 구성품은 본체와 아답터입니다.
​
청담역 직거래 가능하며, 택배는 포장 최고로 해드리나 반품불가 조건으로 가능합니다. 감사합니다.
관심5|채팅1|조회98
0=================================================
맥북 프로 잠금장치 - MACIN Macbook pro (맥킨2세대) + 켄싱턴 락
/images?image_index=0&object_id=451688581&object_type=Article
동작구 흑석동
끌올1시간전
44.8°C
16,000원
54,000 ➞ 29,000
거의 새상품급
쇼핑몰 상품 안내 참조 (사용 가능 모델 및 체결 방법 확인)
 (운영 정책에 의한 URL 자동 숨김) 
🌾 직거래 👨🏻‍🌾
🥕 9호선 흑석역
🥕 중앙대학교병원
🥕 흑석동 구립큰별어린이집
관심2|채팅0|조회76
1=================================================
맥북 프로 M1 8GB 256GB (Macbook Pro M1)
/images?image_index=0&object_id=475620502&object_type=Article
수원시 영통구 영통1동
2시간전
36.9°C
980,000원
2020년 말에 구매한 제품입니다. 미국에서 구매한 제품이라 키보드는 영어로만 되어있습니다. 충전기는 다른 맥북 

KeyboardInterrupt: 

In [85]:
article_id_list[12]

'420032176'

In [101]:
count = 0
for row in range(len(df)) :
    check_point = 0
    content = df.loc[row,'content']
    # print('===')
    try : 
        if '256' in content :
            check_point += 1
            idx = content.find('256')
            end = content[idx:].find('\n')
            text = content[idx:idx+end]
        if '512' in content :
            check_point += 1
            idx = content.find('512')
            end = content[idx:].find('\n')
            text = content[idx:idx+end]
        
        if check_point==1 :
            print(text)
            count += 1
    except : continue
print(count)

256GB - 초고속하드
512SSD 
512GB SSD
256GB 

512GB / 16GB(RAM)
512GB 
512GB 팝니다!
512SSD, i7 10세대(최대 4.1GHz Turbo Boost)
512GB / 16GB(RAM)

512GB
512SSD, i7 10세대(최대 4.1GHz Turbo Boost)
512GB
256 기가바이트 입니다
512ssd
256GB,
256GB
512 스페이스그레이 2021(애플케어플 2024 5 유)
256

512GB
512GB SDD
512
256GB
512모델 판매 합니다
512 모델입니다
256gb SSD
256기가 SSD하드
512 HDD
512 HDD
512GB  SSD
512 HDD
256기가입니다.
256GB
512GB 
512
256
256GB SSD 탑재한 모델입니다.
512GB 입니다!
512GB
256G 저장용량
512GB SSD 저장장치

512GB
256gb)
512GB, 메모리 16GB
512g
256GB 
512기가 ssd
512g 10core gpu 고급형 모델
512GB SSD (MKGR3KH/A)
512GB
256GB
256ssd 풀박스
512GB
512GB PCle
256기가+@추가금으로  맥북프로 m1 교환합니다
256GB는 금액차이없어요.
2560x1600
512GB
256GB
512기가에 램 32기가 10코어입니다
512GB
256GB 입니다!
256기가 저장용량
256G
512gb
256 ssd
512 GB PCIe

256ssd 입니다.
512, 램32). 13인치를 더 자주 사용해서 정리합니다.
256 SSD
256GB 8GB
256gb, ram 16gb
256GB
256기가입니다. 안쪽 터치 패드도 필름 붙여놨고 케이스도있어요. 같이 드립니다. 연락주세요.
256기가



512ssd
512G로 업그레이드 한 제품입니다.
256기가
256기가
512기가
256기가입니다. 안쪽 터치 패드도 필름 붙여놨고 케이스도있어요. 같이 드립니다. 연락주세요.
256 SSD
512

In [14]:
# article_id 까지 해서 관계형 테이블로 만들면 됨
# import pandas as pd

# df = pd.DataFrame(columns=['article_id','title','region','written_date','manner_temperature','price','content','chat_count'])

try :
    driver = webdriver.Chrome(executable_path='/Users/june/Desktop/danggn/chromedriver')
    url = f'https://www.daangn.com/articles/464885104'
    driver.get(url)
    # driver.implicitly_wait(3)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # print(soup.prettify) # 전체 소스코드 훑어보기
    
    ##### parsing
    title = soup.find('h1',attrs={'id':'article-title'}).get_text() # <h1 id="article-title" property="schema:name" style="margin-top:0px;">맥북 프로 MacBook Pro 13(Retina, 2013년 후반 모델)</h1>
    content = soup.find('div',attrs={'id':'article-detail'}).get_text()[1:-1] # <div id="article-detail" property="schema:description">
    region = soup.find('div', attrs={'id':'region-name'}).get_text() # <div id="region-name">강서구 등촌동</div>
    written_date = soup.find('time').get_text().replace(' ','').replace('\n','') # <section id="article-description"> <p id="article-category"> 디지털기기 ∙ <time> 끌올 6시간 전 </time> </p>
    manner_temperature = soup.select_one('#temperature-wrap > dd').get_text().replace(' ','').replace('\n','')
    try : 
        price = soup.find('p',attrs={'id':'article-price'}).get_text().replace(' ','').replace('\n','') # <p content="350000.0" id="article-price" property="schema:price" style="font-size:18px; font-weight:bold;"> 350,000원 </p>
    except :
        price = soup.find('p',attrs={'id':'article-price-nanum'}).get_text().replace(' ','').replace('\n','')
    chat_count = soup.find('p',attrs={'id':'article-counts'}).get_text().replace(' ','').replace('\n','').replace('∙','|') # <p id="article-counts">

    # print(soup.select_one('#slick-slide04 > div > a').attrs['href'])
    print(soup.select_one('#image-slider > div > div > div > div > div > div > a').attrs['href'])
    # print(title)
    # print(region)
    # print(written_date)
    # print(manner_temperature)
    # print(price)
    # print(content)
    # print(chat_count)
    print(f'{c}=================================================')
    
    # df.loc[len(df)] = [article_id,title,region,written_date,manner_temperature,price,content,chat_count]


finally :
    driver.quit()

/var/folders/sd/c66x6jh14991sn8g9gv1f3x80000gn/T/ipykernel_11222/2664769537.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/june/Desktop/danggn/chromedriver')


/images?image_index=8&object_id=464885104&object_type=Article
446=================================================


In [93]:
df.to_csv('당근크롤링.csv',encoding='utf-8-sig',index=False)
df.to_pickle('당근크롤링_피클.pkl')

In [95]:
df2 = pd.read_pickle('당근크롤링_피클.pkl')
df2

,article_id,title,region,written_date,manner_temperature,price,content,chat_count
0,473709426,0.9kg 맥북 골드 가볍다 빠르다 얇다 미니멀의 최고 노트북 애플 한정판 macb...,성동구 행당동,끌올10시간전,56.3°C,"749,000원","애플(APPLE)을 모르는 분들이 있으리라 생각되어\n먼저 말하자면, 가볍지만 고급...",관심8|채팅0|조회218
1,472292418,"Macbook Pro( 맥북 프로)16인치 2019 (i7, 16GB, 512)",광진구 중곡동,끌올10시간전,40.8°C,140만원,Macbook Pro( 맥북 프로)16인치 2019년형\n집에서만 써서 상태 좋아요...,관심2|채팅1|조회109
2,467208356,Macbook air 맥북 에어 M1 2020 dec 256기가,경주시 용강동,끌올10시간전,40.7°C,"870,000원",2020년 12월 생산모델 M1\n전면 힐링쉴드 붙여져있음(액정제외) +4만원\n현...,관심1|채팅1|조회222
3,474643745,MacBook Pro M1 맥북 프로 14인치 512GB (7월 구입 거의 새상품)...,용산구 이태원제2동,11시간전,36.5°C,200만원,M1 프로 칩 / 8코어 CPU / 14코어 GPU / 16GB 메모리 / 512G...,관심2|채팅0|조회48
4,474672466,맥북 프로 macbook pro late 2013 레티나 15.4인치 (RAM16G...,동작구 사당제1동,10시간전,38.6°C,"400,000원",맥북 프로 late 2013 레티나 15.4인치 (RAM 16GB / SSD 500...,관심3|채팅0|조회52
...,...,...,...,...,...,...,...,...
1175,473103945,맥북 프로 14형 M1 미개봉 새제품 팝니다,강서구 등촌동,끌올1일전,36.5°C,215만원,"미사용 새제품 입니다.\n맥북 프로 14인치\nM1 pro 칩, 8Core, 14C...",관심7|채팅1|조회101
1176,470014207,맥북 에어 키스킨 판매합니당,구로구 오류동,끌올1일전,40.6°C,"5,000원",쿠팡에서 구매했어요!\n사용하지 않은 새것이에요\n(박스가 있는데 훼손이 좀 있어요 ㅜ),관심2|채팅2|조회69
1177,470870236,맥북프로 M1 13인치 스페이스그레이,성남시 중원구 도촌동,끌올1일전,37.2°C,103만원,"집에서 서칭, 인강 등으로 가볍게만 사용했고 \n기본에 풀박입니다 -! 충전기 상태...",관심4|채팅2|조회168
1178,456772571,맥북 프로 M1 14형 팝니다.,강서구 내발산동,끌올1일전,40.3°C,220만원,"M1 pro 칩, 8core cpu, 14core gpu, 16GB ram, 512...",관심4|채팅3|조회361
